# OpenRemote: Energy Transition

## Intro

### Technologies used

## Imports

In [99]:
import warnings
import datetime as dt
import json
import requests
import pandas as pd
from pytz import timezone
import numpy as np
import matplotlib.pyplot as plt

In [100]:
PATH = ".\\data\\"

In [101]:
!dir {PATH} #Windows Only Command

 Volume in drive C is Windows
 Volume Serial Number is 984A-EC2B

 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning\data

02/05/2021  19:20    <DIR>          .
02/05/2021  19:20    <DIR>          ..
28/03/2021  20:31            13.803 GetWeatherSiteEstimatedActuals.csv
12/04/2021  10:27            20.994 GetWeatherSiteEstimatedActuals2.csv
23/03/2021  20:55           340.288 SolarData.csv
               3 File(s)        375.085 bytes

 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning


 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning


 Directory of C:\Users\simon\OneDrive - PXL\School\2TINQ\_Research Project\OpenRemote-Energy-Challenge\machine-learning



File Not Found


## API Calls for datasets

In [102]:
API_URL = "https://api.openuv.io/api/v1/forecast"
API_KEY = input("API-key: ")
PREDICTION_PERIOD = 30 #Minutes
TIME_ZONE = timezone("Europe/Amsterdam")

API-key: bc9f0600d36c131f75def897cd96083a 


In [103]:
# OpenRemote Building Location
LAT = '-33.34'
LNG = '115.342'
DT = '2021-02-05 01:08:59.353'

In [104]:
#http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
def get_uv_values(lat, lng, dt):
    try:
        resp = requests.get(API_URL,
            headers={"content-type":"application/json", "x-access-token": API_KEY}, 
            params={'lat': lat, 'lng': lng})
        print("Request sent to API.")
        
        if resp.status_code != 200:     # This means something went wrong
            raise requests.ConnectionError('GET openuv forecast {}'.format(resp.status_code))
        print("Response received.")
        
    except Exception as e:
        print("Could not connect to the API")
        print(str(e))
    
    rows = []
    columns = ['uv', 'uv_time', 'azimuth', 'altitude']

    for data in resp.json():
        for row in resp.json()[data]:
            new_row = [row['uv'], row['uv_time']]
            for key in row['sun_position']:
                new_row.append(row['sun_position'][key])
            rows.append(new_row)

    uv_predictions = pd.DataFrame(rows, columns=columns)
    uv_predictions['uv_time'] = pd.to_datetime(uv_predictions['uv_time'])
    uv_predictions['uv_time'] = pd.DatetimeIndex(pd.to_datetime(uv_predictions.uv_time, utc=True)).tz_convert(TIME_ZONE)
    return uv_predictions

In [105]:
uv_predictions = get_uv_values(LAT, LNG, DT)
uv_predictions

Request sent to API.
Response received.


,uv,uv_time,azimuth,altitude
0,0.0000,2021-05-05 00:56:28.389000+02:00,-1.902090,-0.011366
1,0.2556,2021-05-05 01:56:28.389000+02:00,-2.054485,0.189278
2,1.0615,2021-05-05 02:56:28.389000+02:00,-2.233479,0.372987
3,2.3098,2021-05-05 03:56:28.389000+02:00,-2.452317,0.529805
4,3.4402,2021-05-05 04:56:28.389000+02:00,-2.721193,0.645301
5,4.2953,2021-05-05 05:56:28.389000+02:00,-3.035095,0.702423
6,4.0790,2021-05-05 06:56:28.389000+02:00,2.920365,0.689581
7,3.0372,2021-05-05 07:56:28.389000+02:00,2.620512,0.609605
8,1.8282,2021-05-05 08:56:28.389000+02:00,2.370289,0.477316
9,0.6782,2021-05-05 09:56:28.389000+02:00,2.167438,0.309396


In [106]:
solar_data = pd.read_csv(f"{PATH}SolarData.csv", parse_dates=["timestamp"])

## Handy functions

In [107]:
def display_all(df):
    with pd.option_context("display.max_rows", 10000, "display.max_columns", 10000): 
        display(df)

## Machine learning

### Variables

### Cleaning the data

In [108]:
solar_data #OpenRemote Data

,timestamp,name,attribute_name,value
0,2021-02-11 04:10:22.926,Tamar building,totalPower,0.0
1,2021-02-11 04:10:23.089,Humber building,totalPower,0.0
2,2021-02-25 04:13:48.669,Humber building,totalPower,0.0
3,2021-02-25 04:13:54.488,Tamar building,totalPower,0.0
4,2021-02-11 04:40:23.122,Tamar building,totalPower,0.0
...,...,...,...,...
5469,2021-02-25 02:43:54.267,Tamar building,totalPower,0.0
5470,2021-02-25 03:13:48.480,Humber building,totalPower,0.0
5471,2021-02-25 03:13:54.344,Tamar building,totalPower,0.0
5472,2021-02-25 03:43:48.586,Humber building,totalPower,0.0


Everything we got from the Solcast API:  
  
**Global Horizontal Irradiance** (`Ghi`) is the total solar radiation on a horizontal surface. It is the sum of DNI & DHI  
**Direct Normal Irradiance** (`Dni`) is the amount of solar radiation received per unit area by a surface that is always held in a straight line from the sun's current position  
**Diffuse Horizontal Irradiance** (`Dhi`) is the radiation at the Earth's surface from light scattered by the atmosphere.  
**Direct Beam Horizontal Irradiance** (`Ebh`) The horizontal component of DNI  
**Zenith** is the angle between the sun's rays and the vertical direction.  
**Azimuth** is the angle between the projection of sun's center onto the horizontal plane and due south direction.  
  
*10 and 90 fields represent a 10% and 90% probability bound, around a median forecast value*


In [109]:
uv_predictions # Data from openuv

,uv,uv_time,azimuth,altitude
0,0.0000,2021-05-05 00:56:28.389000+02:00,-1.902090,-0.011366
1,0.2556,2021-05-05 01:56:28.389000+02:00,-2.054485,0.189278
2,1.0615,2021-05-05 02:56:28.389000+02:00,-2.233479,0.372987
3,2.3098,2021-05-05 03:56:28.389000+02:00,-2.452317,0.529805
4,3.4402,2021-05-05 04:56:28.389000+02:00,-2.721193,0.645301
5,4.2953,2021-05-05 05:56:28.389000+02:00,-3.035095,0.702423
6,4.0790,2021-05-05 06:56:28.389000+02:00,2.920365,0.689581
7,3.0372,2021-05-05 07:56:28.389000+02:00,2.620512,0.609605
8,1.8282,2021-05-05 08:56:28.389000+02:00,2.370289,0.477316
9,0.6782,2021-05-05 09:56:28.389000+02:00,2.167438,0.309396


In [110]:
totalPower = solar_data.loc[solar_data['attribute_name'] == 'totalPower'].sort_values(by=['timestamp'], ascending=True).drop(columns=['attribute_name'])

In [111]:
totalPower

,timestamp,name,value
749,2021-02-05 01:08:59.353,Humber building,0.000000
750,2021-02-05 01:08:59.592,Tamar building,0.000000
767,2021-02-05 01:38:59.518,Humber building,0.000000
776,2021-02-05 01:38:59.678,Tamar building,0.000000
811,2021-02-05 02:08:59.722,Humber building,0.000000
...,...,...,...
3247,2021-03-08 11:16:29.019,Tamar building,12.726667
3256,2021-03-08 11:46:28.730,Humber building,2.689000
3257,2021-03-08 11:46:29.236,Tamar building,11.336000
3272,2021-03-08 12:16:28.843,Humber building,3.391500


In [112]:
humber_totalPower = totalPower.loc[totalPower['name'] == 'Humber building'].drop(columns=['name'])
tamar_totalPower = totalPower.loc[totalPower['name'] == 'Tamar building'].drop(columns=['name'])

In [113]:
tamar_totalPower, humber_totalPower

(                   timestamp      value
 750  2021-02-05 01:08:59.592   0.000000
 776  2021-02-05 01:38:59.678   0.000000
 812  2021-02-05 02:09:00.184   0.000000
 816  2021-02-05 02:39:00.276   0.000000
 839  2021-02-05 03:09:00.357   0.000000
 ...                      ...        ...
 3221 2021-03-08 10:16:28.661   6.579666
 3235 2021-03-08 10:46:28.926  11.877333
 3247 2021-03-08 11:16:29.019  12.726667
 3257 2021-03-08 11:46:29.236  11.336000
 3273 2021-03-08 12:16:29.333  15.811334
 
 [1465 rows x 2 columns],
                    timestamp     value
 749  2021-02-05 01:08:59.353  0.000000
 767  2021-02-05 01:38:59.518  0.000000
 811  2021-02-05 02:08:59.722  0.000000
 815  2021-02-05 02:38:59.931  0.000000
 840  2021-02-05 03:09:00.400  0.000000
 ...                      ...       ...
 3220 2021-03-08 10:16:27.919  1.498000
 3234 2021-03-08 10:46:28.337  2.603000
 3246 2021-03-08 11:16:28.467  2.955333
 3256 2021-03-08 11:46:28.730  2.689000
 3272 2021-03-08 12:16:28.843  3.391500


In [114]:
totalPower

,timestamp,name,value
749,2021-02-05 01:08:59.353,Humber building,0.000000
750,2021-02-05 01:08:59.592,Tamar building,0.000000
767,2021-02-05 01:38:59.518,Humber building,0.000000
776,2021-02-05 01:38:59.678,Tamar building,0.000000
811,2021-02-05 02:08:59.722,Humber building,0.000000
...,...,...,...
3247,2021-03-08 11:16:29.019,Tamar building,12.726667
3256,2021-03-08 11:46:28.730,Humber building,2.689000
3257,2021-03-08 11:46:29.236,Tamar building,11.336000
3272,2021-03-08 12:16:28.843,Humber building,3.391500


#### Setting the dataframe to the same TIME_ZONE

In [115]:
totalPower['timestamp'] = totalPower['timestamp'].dt.tz_localize(TIME_ZONE)

In [116]:
# Cleaned DataFrames:
uv_predictions, totalPower

(        uv                          uv_time   azimuth  altitude
 0   0.0000 2021-05-05 00:56:28.389000+02:00 -1.902090 -0.011366
 1   0.2556 2021-05-05 01:56:28.389000+02:00 -2.054485  0.189278
 2   1.0615 2021-05-05 02:56:28.389000+02:00 -2.233479  0.372987
 3   2.3098 2021-05-05 03:56:28.389000+02:00 -2.452317  0.529805
 4   3.4402 2021-05-05 04:56:28.389000+02:00 -2.721193  0.645301
 5   4.2953 2021-05-05 05:56:28.389000+02:00 -3.035095  0.702423
 6   4.0790 2021-05-05 06:56:28.389000+02:00  2.920365  0.689581
 7   3.0372 2021-05-05 07:56:28.389000+02:00  2.620512  0.609605
 8   1.8282 2021-05-05 08:56:28.389000+02:00  2.370289  0.477316
 9   0.6782 2021-05-05 09:56:28.389000+02:00  2.167438  0.309396
 10  0.1179 2021-05-05 10:56:28.389000+02:00  1.999732  0.118637,
                             timestamp             name      value
 749  2021-02-05 01:08:59.353000+01:00  Humber building   0.000000
 750  2021-02-05 01:08:59.592000+01:00   Tamar building   0.000000
 767  2021-02-05 0

In [172]:
#Test loop - Full loop could take +- 10 mins to run!
totalPower_uv = []
totalPower_azimuth = []
totalPower_altitude = []

for i, totalPower_row in totalPower.tail().iterrows():
    
    #uv_predictions = get_uv_values(LAT, LNG, '2021-03-08 11:16:29.019000')
    closest_date = min(uv_predictions.uv_time, key=lambda x: abs(x - totalPower_row.timestamp))
    closest_date_row = uv_predictions.loc[uv_predictions.uv_time == closest_date]
    
    totalPower_uv.append(closest_date_row['uv'][0])
    print('Given nr', i, 'an uv value of', closest_date_row['uv'][0])
    totalPower_azimuth.append(closest_date_row['azimuth'][0])
    totalPower_altitude.append(closest_date_row['altitude'][0])
    
print(type(pd.Series(totalPower_uv, dtype='float64')[0]))
totalPower['uv'] = pd.Series(totalPower_uv, dtype='float64')
totalPower['azimuth'] = pd.Series(totalPower_azimuth, dtype='float64')
totalPower['altitude'] = pd.Series(totalPower_altitude, dtype='float64')

Given nr 3247 an uv value of 0.0
Given nr 3256 an uv value of 0.0
Given nr 3257 an uv value of 0.0
Given nr 3272 an uv value of 0.0
Given nr 3273 an uv value of 0.0
<class 'numpy.float64'>


In [173]:
totalPower = totalPower.sort_values(by=['uv'])

In [174]:
totalPower.uv

0       0.0
1       0.0
4       0.0
2       0.0
3       0.0
       ... 
3247    NaN
3256    NaN
3257    NaN
3272    NaN
3273    NaN
Name: uv, Length: 2939, dtype: float64